In [1]:
pip install -U pandasql

Requirement already up-to-date: pandasql in c:\users\crie072\anaconda3\lib\site-packages (0.7.3)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import csv
import xlrd
import io
from zipfile import ZipFile
from datetime import date, timedelta, time, datetime

import glob

import argparse
import logging
import os

from pathlib import Path

from pandasql import sqldf

In [3]:
today_yyyymmdd = date.today().strftime("%Y%m%d")
print(today_yyyymmdd)

20200916


In [4]:
#pandasの表示列数を増やす
import pandas as pd
pd.set_option('display.max_columns', 100)

In [5]:
data_dir = Path(r"C:\Users\crie072\Documents\21_Online")

入会データ取込

In [6]:
path = r"C:\Users\crie072\Documents\01_データ関連\Adhoc\202008_N29" # use your path
all_files = glob.glob(path + "/*.xlsx")

li = []
 
for filename in all_files:
    df = pd.read_excel(filename)
    li.append(df)
df_all = pd.concat(li, axis=0, ignore_index=True)

ファイル名設定

In [7]:
output = "N29オンライン入会承認累積"
output2 = "デジガレアフィリエイト申込累積（通常）"
output3 = "デジガレアフィリエイト申込累積（利用条件付き）"

In [8]:
print(df_all.dtypes)

カード加入日           int64
会員カード発行区分       object
続柄               int64
カード提携会社1CD      object
カード提携会社2CD       int64
カード種類CD          int64
カードブランドCD        int64
切替CD             int64
勧誘CD             int64
受付部支店            int64
性別CD             int64
再発行CD            int64
旧カード提携会社1CD     object
旧カード提携会社2CD    float64
旧カード種類CD       float64
旧カードブランドCD     float64
入会申込番号           int64
満年齢              int64
リボ支払方法CD       float64
勧誘団体CD           int64
提携カード特殊エリア     float64
エンボス内容         float64
カード年会費CD         int64
リボ利用対象CD         int64
リボ利用対象CD.1     float64
dtype: object


In [9]:
df_app =df_all[((df_all["会員カード発行区分"] == 1) | (df_all["会員カード発行区分"] == "1"))
               & (df_all["続柄"] == 0) 
               & ~(df_all["カード提携会社2CD"] == 95) 
               & (df_all["切替CD"] == 0) 
               & (df_all["受付部支店"] == 299)]
df_app

,カード加入日,会員カード発行区分,続柄,カード提携会社1CD,カード提携会社2CD,カード種類CD,カードブランドCD,切替CD,勧誘CD,受付部支店,性別CD,再発行CD,旧カード提携会社1CD,旧カード提携会社2CD,旧カード種類CD,旧カードブランドCD,入会申込番号,満年齢,リボ支払方法CD,勧誘団体CD,提携カード特殊エリア,エンボス内容,カード年会費CD,リボ利用対象CD,リボ利用対象CD.1
0,20200106,1,0,D00,8,31,1,0,3,299,2,0,NaN,NaN,NaN,NaN,155051921622839,56,12.0,6000,4.297253e+17,NaN,6000,2,0.0
1,20200106,1,0,D00,4,31,2,0,3,299,2,0,NaN,NaN,NaN,NaN,155051921586919,42,12.0,7000,4.297250e+17,NaN,7000,2,0.0
2,20200106,1,0,D00,1,31,1,0,3,299,2,0,NaN,NaN,NaN,NaN,155051921619298,50,52.0,4000,4.000000e+01,NaN,4000,2,1.0
3,20200106,1,0,D00,8,31,1,0,3,299,1,0,NaN,NaN,NaN,NaN,155051921626640,50,12.0,6000,4.000000e+01,NaN,6000,2,0.0
4,20200106,1,0,D00,1,2,1,0,3,299,2,0,NaN,NaN,NaN,NaN,155051921627812,38,12.0,11000,4.000000e+01,NaN,11000,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34751,20200831,1,0,D00,1,31,1,0,3,299,2,0,NaN,NaN,NaN,NaN,155051921970709,24,52.0,12000,4.000000e+01,12000.0,2,1,NaN
34752,20200831,1,0,D00,1,31,1,0,3,299,1,0,NaN,NaN,NaN,NaN,155051921966566,45,12.0,14000,4.000000e+01,14000.0,2,0,NaN
34753,20200831,1,0,D00,1,31,1,0,3,299,2,0,NaN,NaN,NaN,NaN,155051921966186,28,12.0,6000,4.000000e+01,6000.0,2,0,NaN
34756,20200831,1,0,D00,1,31,1,0,3,299,2,0,NaN,NaN,NaN,NaN,155051921960254,60,12.0,6000,4.000000e+01,6000.0,2,0,NaN


In [10]:
_df_app = df_app[['カード加入日','性別CD','入会申込番号','満年齢']]
_df_app.head()

,カード加入日,性別CD,入会申込番号,満年齢
0,20200106,2,155051921622839,56
1,20200106,2,155051921586919,42
2,20200106,2,155051921619298,50
3,20200106,1,155051921626640,50
4,20200106,2,155051921627812,38


In [11]:
#重複削除
_df_app = _df_app.drop_duplicates(subset=['入会申込番号'])
_df_app.shape[0]

5149

In [12]:
#df_tmp = _df_app.copy()
_df_app['年代'] = _df_app['満年齢']//10 * 10
_df_app.head()

,カード加入日,性別CD,入会申込番号,満年齢,年代
0,20200106,2,155051921622839,56,50
1,20200106,2,155051921586919,42,40
2,20200106,2,155051921619298,50,50
3,20200106,1,155051921626640,50,50
4,20200106,2,155051921627812,38,30


In [15]:
_df_app['カード加入日']=pd.to_datetime(_df_app['カード加入日'], format='%Y%m%d')
#"カード加入年月"のカラム生成
_df_app['カード加入年'] = _df_app['カード加入日'].dt.year
_df_app['カード加入月'] = _df_app['カード加入日'].dt.month

_df_app['カード加入年月'] = _df_app[['カード加入年', 'カード加入月']].apply(lambda x: '{}年{}月'.format(x[0], x[1]), axis=1)
_df_app = _df_app.drop(columns=['カード加入年','カード加入月'])

_df_app['カード加入日']= _df_app['カード加入日'].dt.strftime('%Y/%m/%d')
_df_app['承認有無'] = 1
_df_app.head()

,カード加入日,性別CD,入会申込番号,満年齢,年代,カード加入年月,承認有無
0,2020/01/06,2,155051921622839,56,50,2020年1月,1
1,2020/01/06,2,155051921586919,42,40,2020年1月,1
2,2020/01/06,2,155051921619298,50,50,2020年1月,1
3,2020/01/06,1,155051921626640,50,50,2020年1月,1
4,2020/01/06,2,155051921627812,38,30,2020年1月,1


In [16]:
#CSVアウトプット
_df_app.to_csv(data_dir /f"{output}.csv", index=False,encoding="shift_jis")

In [17]:
df_count= _df_app.groupby(['カード加入年月'])['承認有無'].count()
df_count

カード加入年月
2020年1月     737
2020年2月    1056
2020年3月     388
2020年4月     116
2020年5月     585
2020年6月     697
2020年7月     889
2020年8月     681
Name: 承認有無, dtype: int64

In [18]:
pd.crosstab(_df_app['カード加入年月'],_df_app['年代'])

年代,10,20,30,40,50,60,70,80
カード加入年月,,,,,,,,
2020年1月,2,91,186,184,173,76,23,2
2020年2月,3,99,271,305,228,108,40,2
2020年3月,0,54,98,107,77,42,10,0
2020年4月,2,23,35,26,17,10,3,0
2020年5月,2,65,149,187,118,46,15,3
2020年6月,5,100,177,188,129,77,20,1
2020年7月,3,129,228,246,180,81,20,2
2020年8月,1,93,168,204,144,58,10,3


デジガレリスト取込

In [20]:
data_dir = Path(r"C:\Users\crie072\Documents\21_Online")
input_file = "デジガレアフィリエイト申込累積.xlsx"

In [21]:
#シート名取得
bk = pd.ExcelFile(data_dir/r"input"/f"{input_file}")
print(bk.sheet_names)

['統合（通常）', '統合（利用条件付き）']


In [22]:
df_dg = pd.read_excel(data_dir/r"input"/f"{input_file}", sheet_name = 0)
df_dg

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価
0,37.0,げん玉,ポイント,2020-01-08 18:43:29,2020-01-08 18:56:06,155051921643363,BGT,9000
1,37.0,げん玉,ポイント,2020-01-08 19:11:08,2020-01-08 19:32:05,155051921643397,BGT,9000
2,82.0,ちょびリッチ。ドットコム,ポイント,2020-01-08 20:07:13,2020-01-08 20:18:06,155051921643538,BGT,9000
3,52.0,ECナビSP,ポイント,2020-01-08 20:21:44,2020-01-08 20:31:05,155051921643553,BGT,9000
4,52.0,ECナビSP,ポイント,2020-01-08 20:31:50,2020-01-08 20:39:06,155051921643587,BGT,9000
...,...,...,...,...,...,...,...,...
4564,NaN,モッピー,ポイント,2020-06-29 00:00:00,2020-06-29 00:00:00,155051921940645,BGT,7000
4565,NaN,ハピタス,ポイント,2020-05-08 00:00:00,2020-05-08 00:00:00,155051921880270,BGT,7000
4566,NaN,ハピタス,ポイント,2020-07-04 00:00:00,2020-07-04 00:00:00,155980006515510,BGT,7000
4567,NaN,モッピー,ポイント,2020-06-27 00:00:00,2020-06-27 00:00:00,155051921937450,BGT,7000


In [23]:
#ASIDの整数化
for i in range(len(df_dg)):
    if pd.isnull(df_dg['ASID'][i]) == True:
        df_dg.loc[:,'ASID'][i] = 0
df_dg = df_dg.astype({'ASID':int})

C:\Users\crie072\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [24]:
#重複削除
df_dg = df_dg.drop_duplicates(subset=['参照用'])
df_dg.shape[0]

3925

In [25]:
#"申込年月"のカラム生成
df_dg['申込年'] = df_dg['申込'].dt.year
df_dg['申込月'] = df_dg['申込'].dt.month

df_dg['申込年月'] = df_dg[['申込年', '申込月']].apply(lambda x: '{}年{}月'.format(x[0], x[1]), axis=1)
df_dg = df_dg.drop(columns=['申込年','申込月'])
df_dg

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,申込年月
0,37,げん玉,ポイント,2020-01-08 18:43:29,2020-01-08 18:56:06,155051921643363,BGT,9000,2020年1月
1,37,げん玉,ポイント,2020-01-08 19:11:08,2020-01-08 19:32:05,155051921643397,BGT,9000,2020年1月
2,82,ちょびリッチ。ドットコム,ポイント,2020-01-08 20:07:13,2020-01-08 20:18:06,155051921643538,BGT,9000,2020年1月
3,52,ECナビSP,ポイント,2020-01-08 20:21:44,2020-01-08 20:31:05,155051921643553,BGT,9000,2020年1月
4,52,ECナビSP,ポイント,2020-01-08 20:31:50,2020-01-08 20:39:06,155051921643587,BGT,9000,2020年1月
...,...,...,...,...,...,...,...,...,...
4564,0,モッピー,ポイント,2020-06-29 00:00:00,2020-06-29 00:00:00,155051921940645,BGT,7000,2020年6月
4565,0,ハピタス,ポイント,2020-05-08 00:00:00,2020-05-08 00:00:00,155051921880270,BGT,7000,2020年5月
4566,0,ハピタス,ポイント,2020-07-04 00:00:00,2020-07-04 00:00:00,155980006515510,BGT,7000,2020年7月
4567,0,モッピー,ポイント,2020-06-27 00:00:00,2020-06-27 00:00:00,155051921937450,BGT,7000,2020年6月


In [26]:
df_dg2 = pd.merge(df_dg,_df_app, left_on =['参照用'], right_on = ['入会申込番号'], how = 'left')
df_dg2 = df_dg2.drop(columns={'入会申込番号'})
df_dg2.head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,申込年月,カード加入日,性別CD,満年齢,年代,カード加入年月,承認有無
0,37,げん玉,ポイント,2020-01-08 18:43:29,2020-01-08 18:56:06,155051921643363,BGT,9000,2020年1月,2020/02/04,1.0,34.0,30.0,2020年2月,1.0
1,37,げん玉,ポイント,2020-01-08 19:11:08,2020-01-08 19:32:05,155051921643397,BGT,9000,2020年1月,2020/01/09,1.0,49.0,40.0,2020年1月,1.0
2,82,ちょびリッチ。ドットコム,ポイント,2020-01-08 20:07:13,2020-01-08 20:18:06,155051921643538,BGT,9000,2020年1月,NaN,NaN,NaN,NaN,NaN,NaN
3,52,ECナビSP,ポイント,2020-01-08 20:21:44,2020-01-08 20:31:05,155051921643553,BGT,9000,2020年1月,2020/01/22,1.0,30.0,30.0,2020年1月,1.0
4,52,ECナビSP,ポイント,2020-01-08 20:31:50,2020-01-08 20:39:06,155051921643587,BGT,9000,2020年1月,2020/01/14,1.0,26.0,20.0,2020年1月,1.0


In [27]:
df_dg2 = df_dg2.sort_values(['申込','参照用'])

In [28]:
for i in range(len(df_dg2)):
    if pd.isnull(df_dg2['性別CD'][i]) == True:
        df_dg2.loc[:,'性別CD'][i] = 0
        
df_dg2 = df_dg2.astype({'性別CD':int})

for i in range(len(df_dg2)):
    if pd.isnull(df_dg2['満年齢'][i]) == True:
        df_dg2.loc[:,'満年齢'][i] = 0
        
df_dg2 = df_dg2.astype({'満年齢':int})

for i in range(len(df_dg2)):
    if pd.isnull(df_dg2['承認有無'][i]) == True:
        df_dg2.loc[:,'承認有無'][i] = 0
        
df_dg2 = df_dg2.astype({'承認有無':int})

for i in range(len(df_dg2)):
    if pd.isnull(df_dg2['年代'][i]) == True:
        df_dg2.loc[:,'年代'][i] = 0
        
df_dg2 = df_dg2.astype({'年代':int})

df_dg2.head()

C:\Users\crie072\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\crie072\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\crie072\anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,申込年月,カード加入日,性別CD,満年齢,年代,カード加入年月,承認有無
0,37,げん玉,ポイント,2020-01-08 18:43:29,2020-01-08 18:56:06,155051921643363,BGT,9000,2020年1月,2020/02/04,1,34,30,2020年2月,1
1,37,げん玉,ポイント,2020-01-08 19:11:08,2020-01-08 19:32:05,155051921643397,BGT,9000,2020年1月,2020/01/09,1,49,40,2020年1月,1
2,82,ちょびリッチ。ドットコム,ポイント,2020-01-08 20:07:13,2020-01-08 20:18:06,155051921643538,BGT,9000,2020年1月,NaN,0,0,0,NaN,0
3,52,ECナビSP,ポイント,2020-01-08 20:21:44,2020-01-08 20:31:05,155051921643553,BGT,9000,2020年1月,2020/01/22,1,30,30,2020年1月,1
4,52,ECナビSP,ポイント,2020-01-08 20:31:50,2020-01-08 20:39:06,155051921643587,BGT,9000,2020年1月,2020/01/14,1,26,20,2020年1月,1


In [29]:
pd.crosstab(df_dg2['申込年月'],df_dg2['カード加入年月'])

カード加入年月,2020年1月,2020年2月,2020年3月,2020年4月,2020年5月,2020年6月,2020年7月,2020年8月
申込年月,,,,,,,,
2020年1月,449,536,45,4,0,0,0,0
2020年2月,0,242,132,4,2,2,0,0
2020年3月,0,0,43,13,3,5,0,0
2020年4月,0,0,0,20,11,5,0,0
2020年5月,0,0,0,0,192,145,10,3
2020年6月,0,0,0,0,0,172,109,10
2020年7月,0,0,0,0,0,0,206,111
2020年8月,0,0,0,0,0,0,0,86


In [30]:
#CSVアウトプット(通常分)
df_dg2.to_csv(data_dir /f"{output2}.csv", index=False,encoding="shift_jis")

利用条件付き

In [31]:
df_dg_use = pd.read_excel(data_dir/r"input"/f"{input_file}", sheet_name = 1)
df_dg_use

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価
0,37.0,げん玉,ポイント,2020-05-01 00:06:29,2020-05-01 00:22:04,155051921869356,BGT,9000
1,37.0,げん玉,ポイント,2020-05-01 00:31:25,2020-05-01 00:52:05,155051921869372,BGT,9000
2,37.0,げん玉,ポイント,2020-05-01 01:21:40,2020-05-01 01:28:04,155051921869380,BGT,9000
3,21.0,ポイントインカム,ポイント,2020-05-01 06:15:50,2020-05-01 06:25:05,155051921869414,BGT,9000
4,37.0,げん玉,ポイント,2020-05-01 06:27:16,2020-05-01 06:34:04,155051921869422,BGT,9000
...,...,...,...,...,...,...,...,...
2523,5954.0,GetMoney!(JIPC参加企業)（連携用）,ポイント,2020-06-18 04:07:00,2020-06-18 04:15:11,155051921922890,BGT,9000
2524,57.0,ライフメディア,ポイント,2020-06-18 18:44:50,2020-06-18 19:03:12,155051921923690,BGT,9000
2525,37.0,げん玉,ポイント,2020-06-18 20:53:46,2020-06-18 21:10:13,155051921923930,BGT,9000
2526,57.0,ライフメディア,ポイント,2020-06-19 06:45:51,2020-06-19 07:13:12,155051921924177,BGT,9000


In [32]:
#ASIDの整数化
for i in range(len(df_dg_use)):
    if pd.isnull(df_dg_use['ASID'][i]) == True:
        df_dg_use.loc[:,'ASID'][i] = 0
df_dg_use = df_dg_use.astype({'ASID':int})

C:\Users\crie072\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [33]:
#重複削除
df_dg_use = df_dg_use.drop_duplicates(subset=['参照用'])
df_dg_use.shape[0]

1361

In [34]:
#"申込年月"のカラム生成
df_dg_use['申込年'] = df_dg_use['申込'].dt.year
df_dg_use['申込月'] = df_dg_use['申込'].dt.month

df_dg_use['申込年月'] = df_dg_use[['申込年', '申込月']].apply(lambda x: '{}年{}月'.format(x[0], x[1]), axis=1)
df_dg_use = df_dg_use.drop(columns=['申込年','申込月'])
df_dg_use

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,申込年月
0,37,げん玉,ポイント,2020-05-01 00:06:29,2020-05-01 00:22:04,155051921869356,BGT,9000,2020年5月
1,37,げん玉,ポイント,2020-05-01 00:31:25,2020-05-01 00:52:05,155051921869372,BGT,9000,2020年5月
2,37,げん玉,ポイント,2020-05-01 01:21:40,2020-05-01 01:28:04,155051921869380,BGT,9000,2020年5月
3,21,ポイントインカム,ポイント,2020-05-01 06:15:50,2020-05-01 06:25:05,155051921869414,BGT,9000,2020年5月
4,37,げん玉,ポイント,2020-05-01 06:27:16,2020-05-01 06:34:04,155051921869422,BGT,9000,2020年5月
...,...,...,...,...,...,...,...,...,...
2152,37,げん玉,ポイント,2020-08-17 21:34:42,2020-08-17 21:50:08,155051921994238,BGT,9000,2020年8月
2157,0,getmoney,ポイント,2020-07-25 00:00:00,2020-07-25 00:00:00,155051921974289,BGT,9000,2020年7月
2158,0,ちょびリッチ,ポイント,2020-07-07 00:00:00,2020-07-07 00:00:00,155051921955825,BGT,9000,2020年7月
2159,0,ポイントインカム,ポイント,2020-07-24 20:09:00,2020-07-24 00:00:00,155051921973299,BGT,9000,2020年7月


In [35]:
df_dg2_use = pd.merge(df_dg_use,_df_app, left_on =['参照用'], right_on = ['入会申込番号'], how = 'left')
df_dg2_use = df_dg2_use.drop(columns={'入会申込番号'})
df_dg2_use.head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,申込年月,カード加入日,性別CD,満年齢,年代,カード加入年月,承認有無
0,37,げん玉,ポイント,2020-05-01 00:06:29,2020-05-01 00:22:04,155051921869356,BGT,9000,2020年5月,NaN,NaN,NaN,NaN,NaN,NaN
1,37,げん玉,ポイント,2020-05-01 00:31:25,2020-05-01 00:52:05,155051921869372,BGT,9000,2020年5月,2020/06/15,1.0,41.0,40.0,2020年6月,1.0
2,37,げん玉,ポイント,2020-05-01 01:21:40,2020-05-01 01:28:04,155051921869380,BGT,9000,2020年5月,NaN,NaN,NaN,NaN,NaN,NaN
3,21,ポイントインカム,ポイント,2020-05-01 06:15:50,2020-05-01 06:25:05,155051921869414,BGT,9000,2020年5月,2020/06/11,1.0,33.0,30.0,2020年6月,1.0
4,37,げん玉,ポイント,2020-05-01 06:27:16,2020-05-01 06:34:04,155051921869422,BGT,9000,2020年5月,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
df_dg2_use = df_dg2_use.sort_values(['申込','参照用'])

In [37]:
for i in range(len(df_dg2_use)):
    if pd.isnull(df_dg2_use['性別CD'][i]) == True:
        df_dg2_use.loc[:,'性別CD'][i] = 0
        
df_dg2_use = df_dg2_use.astype({'性別CD':int})

for i in range(len(df_dg2_use)):
    if pd.isnull(df_dg2_use['満年齢'][i]) == True:
        df_dg2_use.loc[:,'満年齢'][i] = 0
        
df_dg2_use = df_dg2_use.astype({'満年齢':int})

for i in range(len(df_dg2_use)):
    if pd.isnull(df_dg2_use['承認有無'][i]) == True:
        df_dg2_use.loc[:,'承認有無'][i] = 0
        
df_dg2_use = df_dg2_use.astype({'承認有無':int})

for i in range(len(df_dg2_use)):
    if pd.isnull(df_dg2_use['年代'][i]) == True:
        df_dg2_use.loc[:,'年代'][i] = 0
        
df_dg2_use = df_dg2_use.astype({'年代':int})

df_dg2_use.head()

C:\Users\crie072\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\crie072\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\crie072\anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,申込年月,カード加入日,性別CD,満年齢,年代,カード加入年月,承認有無
1163,0,ライフメディア,NaN,2020-05-01 00:00:00,2020-05-01 00:00:00,155051921869711,BGT,9000,2020年5月,2020/05/28,1,51,50,2020年5月,1
0,37,げん玉,ポイント,2020-05-01 00:06:29,2020-05-01 00:22:04,155051921869356,BGT,9000,2020年5月,NaN,0,0,0,NaN,0
1,37,げん玉,ポイント,2020-05-01 00:31:25,2020-05-01 00:52:05,155051921869372,BGT,9000,2020年5月,2020/06/15,1,41,40,2020年6月,1
2,37,げん玉,ポイント,2020-05-01 01:21:40,2020-05-01 01:28:04,155051921869380,BGT,9000,2020年5月,NaN,0,0,0,NaN,0
3,21,ポイントインカム,ポイント,2020-05-01 06:15:50,2020-05-01 06:25:05,155051921869414,BGT,9000,2020年5月,2020/06/11,1,33,30,2020年6月,1


In [38]:
pd.crosstab(df_dg2_use['申込年月'],df_dg2_use['カード加入年月'])

カード加入年月,2020年5月,2020年6月,2020年7月,2020年8月
申込年月,,,,
2020年5月,250,73,2,0
2020年6月,0,101,52,5
2020年7月,0,0,305,145
2020年8月,0,0,0,119


In [39]:
#CSVアウトプット(利用条件付き)
df_dg2_use.to_csv(data_dir /f"{output3}.csv", index=False,encoding="shift_jis")